## Data Preparation:

Convert the PDF reports to text format using PDF parsing libraries like PyPDF2, pdfplumber, or tools like Apache PDFBox.
Clean and preprocess the extracted text to remove any formatting artifacts, page numbers, headers/footers, etc.
Parse the provided sheet that maps company-specific naming conventions to the standard Activity Metric names (AMKEYs).

In [12]:
import os
import re
import pandas as pd
from PyPDF2 import PdfReader
from datetime import datetime

def parse_pdf(file_path):
    with open(file_path, 'rb') as file:
        pdf_reader = PdfReader(file)
        text = ""
        for page_num in range(len(pdf_reader.pages)):
            text += pdf_reader.pages[page_num].extract_text()
    return text

In [ ]:
def load_metric_mapping(csv_file_path):
    metric_mapping_df = pd.read_csv(csv_file_path)
    # Convert the DataFrame to a dictionary for easy lookup
    metric_keywords = dict(zip(metric_mapping_df['AMKEY'], metric_mapping_df['ActivityMetric']))
    return metric_keywords

def extract_metrics(text, metric_keywords):
    extracted_data = {}
    for metric_key, metric_names in metric_keywords.items():
        for name in metric_names:
            # Using a case-insensitive regex to find metric values
            matches = re.findall(fr"{re.escape(name)}\s*:\s*([\d.]+)", text, re.IGNORECASE)
            if matches:
                extracted_data[metric_key] = float(matches[0])
                break
    return extracted_data

In [ ]:
def process_reports(base_folder, metric_keywords, start_year, end_year):
    result_data = []

    for company_folder in os.listdir(base_folder):
        company_path = os.path.join(base_folder, company_folder)
        if os.path.isdir(company_path):
            for report_year in range(start_year, end_year + 1):
                report_file_path = os.path.join(company_path, f"{report_year}_report.pdf")
                
                if os.path.exists(report_file_path):
                    text = parse_pdf(report_file_path)
                    extracted_data = extract_metrics(text, metric_keywords)
                    result_data.append({
                        'Company': company_folder,
                        'Year': report_year,
                        **extracted_data
                    })

    return result_data


In [ ]:
base_folder = 'annual_reports'
metric_keywords_csv = 'ActivityMetricsSynonyms.csv'  # Path to the CSV file
start_year = 2019
end_year = 2021
result = process_reports(base_folder, metric_keywords_csv, start_year, end_year)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'annual_reports'

In [ ]:
# Convert the result to a DataFrame for further analysis or export
train_df = pd.DataFrame(result)
print(train_df)

Empty DataFrame
Columns: []
Index: []


In [ ]:
KeyError

KeyError

In [ ]:
# Load the Activity Metrics CSV file
activity_metrics_df = pd.read_csv("AMKEY_GoldenStandard.csv")
activity_metrics_df.head(20)

,AMKEY,ActivityMetric
0,3,Advisory fees as per income statement
1,6,Air emissions of the following pollutants: (1) CO
2,7,Air emissions of the following pollutants: (2)...
3,8,Air emissions of the following pollutants: (3)...
4,9,Air emissions of the following pollutants: (4)...
5,10,Air emissions of the following pollutants: (5)...
6,11,ALL Administration expenses per income statement
7,12,All Inury Frequency Rate (Injuries/1m hrs worked)
8,13,"Amount of assets under management, by asset cl..."
9,14,"Amount of assets under management, by asset cl..."


In [ ]:
# Load the second set of Activity Metrics CSV file
activity_metrics_set2 = pd.read_csv("ActivityMetricsSynonyms.csv")
activity_metrics_set2.head(20)

,AMKEY,Group,ActivityMetric,ClientMetric
0,7,Impala,Air emissions of the following pollutants: (2)...,Total indirect Nox
1,7,Sasol,Air emissions of the following pollutants: (2)...,Nitrogen oxides (NOx) (kilotons)
2,8,Sasol,Air emissions of the following pollutants: (3)...,Sulphur oxides (SOx ) (kilotons)
3,8,Ssw,Air emissions of the following pollutants: (3)...,SO2 emissions
4,8,Impala,Air emissions of the following pollutants: (3)...,Total direct SO2 + Total indirect SO2
5,9,Sasol,Air emissions of the following pollutants: (4)...,Particulates (fly ash) (kilotons)
6,10,Sasol,Air emissions of the following pollutants: (5)...,Volatile Organic Compounds (VOC) Indicator of ...
7,12,Tongaat,All Inury Frequency Rate (Injuries/1m hrs worked),Total injury frequency rate (TIFR) – employees...
8,27,Ssw,Area of forestland managed by the entity,Land under management
9,27,Impala,Area of forestland managed by the entity,Leased and managed land - South Africa


In [ ]:
from pdfminer.high_level import extract_text
import os

# Function to extract text from a PDF file using pdfminer
def extract_text_from_pdf(pdf_file):
    text = extract_text(pdf_file)
    return text

# List of PDFs with corresponding IDs and file names
pdf_files = [
    {'ID': 'Oceana1', 'File Name': r'c:\Users\hp\Victor-Files\unify_documents\Multiple source data\Oceana_Group_Sustainability_Report_2022.pdf'},
    {'ID': 'Ssw', 'File Name': r'c:\Users\hp\Victor-Files\unify_documents\Unstructured data\ssw-IR22.pdf'},
    {'ID': 'Sasol', 'File Name': r'c:\Users\hp\Victor-Files\unify_documents\Structured data\SASOL Sustainability Report 2023 20-09_0.pdf'},
    {'ID': 'Impala', 'File Name': r'c:\Users\hp\Victor-Files\unify_documents\Structured data\ESG-spreads.pdf'},
    {'ID': 'Clicks', 'File Name': r'c:\Users\hp\Victor-Files\unify_documents\Structured data\Clicks-Sustainability-Report-2022.pdf'},
    {'ID': 'Oceana2', 'File Name': r'c:\Users\hp\Victor-Files\unify_documents\Multiple source data\Oceana_ESG_Databook_FY2022.pdf'},
    {'ID': 'Uct2', 'File Name': r'c:\Users\hp\Victor-Files\unify_documents\Multiple source data\afs2021.pdf'},
    {'ID': 'Picknpay', 'File Name': r'c:\Users\hp\Victor-Files\unify_documents\Structured data\picknpay-esg-report-spreads-2023.pdf'},
    {'ID': 'Distell', 'File Name': r'c:\Users\hp\Victor-Files\unify_documents\Structured data\DISTELL ESG Appendix 2022.pdf'},
    {'ID': 'Uct1', 'File Name': r'c:\Users\hp\Victor-Files\unify_documents\Multiple source data\UCT_Carbon_Footprint_Report_2020-2021.pdf'},
    {'ID': 'Absa', 'File Name': r'c:\Users\hp\Victor-Files\unify_documents\Structured data\2022-Absa-Group-limited-Environmental-Social-and-Governance-Data-sheet.pdf'},
    {'ID': 'Tongaat', 'File Name': r'c:\Users\hp\Victor-Files\unify_documents\Structured data\2021ESG.pdf'}
]

# Directory to store extracted text files
output_directory = 'extracted_text_pdfminer'

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Loop through each PDF file, extract text, and save to a text file
for pdf_info in pdf_files:
    pdf_id = pdf_info['ID']
    pdf_file_name = pdf_info['File Name']
    
    # Extract text
    extracted_text = extract_text_from_pdf(pdf_file_name)
    
    # Save extracted text to a text file
    output_file_path = os.path.join(output_directory, f'{pdf_id}_extracted_text_pdfminer.txt')
    with open(output_file_path, 'w', encoding='utf-8') as output_file:
        output_file.write(extracted_text)

    print(f'Text extracted from {pdf_id}: {output_file_path}')


FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\hp\\Victor-Files\\unify_documents\\Multiple source data\\Oceana_Group_Sustainability_Report_2022.pdf'

In [ ]:
extract_text

In [ ]:
import pdfplumber
import pandas as pd


# List of annual reports with corresponding file names
annual_reports = [
    ("Oceana1.pdf", "Oceana_Group_Sustainability_Report_2022.pdf"),
    ("Ssw.pdf", "ssw-IR22.pdf"),
    ("Sasol.pdf", "SASOL Sustainability Report 2023 20-09_0.pdf"),
    ("Impala.pdf", "ESG-spreads.pdf"),
    ("Clicks.pdf", "Clicks-Sustainability-Report-2022.pdf"),
    ("Oceana2.pdf", "Oceana_ESG_Databook_FY2022.pdf"),
    ("Uct2.pdf", "afs2021.pdf"),
    ("Picknpay.pdf", "picknpay-esg-report-spreads-2023.pdf"),
    ("Distell.pdf", "DISTELL ESG Appendix 2022.pdf"),
    ("Uct1.pdf", "UCT_Carbon_Footprint_Report_2020-2021.pdf"),
    ("Absa.pdf", "2022-Absa-Group-limited-Environmental-Social-and-Governance-Data-sheet.pdf"),
    ("Tongaat.pdf", "2021ESG.pdf")
]

# Loop through the annual reports and extract text relevant to Activity Metrics
for report_id, file_name in annual_reports:
    with pdfplumber.open(file_name) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text()

        # Extract information only for relevant Activity Metrics
        for _, row in activity_metrics_df.iterrows():
            amkey = row["AMKEY"]
            group = row["Group"]
            activity_metric = row["ActivityMetric"]
            client_metric = row["ClientMetric"]

            # Check if the activity metric information is present in the report
            if amkey in text and group in text and activity_metric in text and client_metric in text:
                # Extract and process the relevant information as needed
                relevant_info = text[text.find(activity_metric):text.find(client_metric)]
                print(f"Document ID: {report_id}, AMKEY: {amkey}")
                print(relevant_info)
                print("=" * 50)


: 